In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os


In [3]:
!pip install langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.6 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.7/306.7 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.0 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: requests-toolbelt
    Found existing installation: requests-toolbelt 0.10.1
    Uninstalling requests-toolbelt-0.10.1:
      Successfully uninstalled requests-toolbelt-0.10.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-bigquery 2.34.4 requires packaging<22.0dev,>=14.

In [4]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 23.0 MB/s eta 0:00:00a 0:00:01


In [5]:
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 3.4 MB/s eta 0:00:00a 0:00:01


In [6]:
from typing import List, Dict, Optional
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader, PyPDFLoader
from langchain.tools import Tool
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoModelForSequenceClassification,
    pipeline
)
import torch
from sentence_transformers import SentenceTransformer
import requests
from bs4 import BeautifulSoup
import os
from datetime import datetime

In [7]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.6 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of opentelemetry-sdk to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.5/615.5 kB 10.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 43.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 48.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 67.5 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [8]:
!pip install chromadb


In [9]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from sentence_transformers import SentenceTransformer
import torch
import chromadb
import re

# Initialize ChromaDB client and embedding model
client = chromadb.Client()
model = SentenceTransformer('all-mpnet-base-v2')

class ThreatIntelRAG:
    def __init__(self, threat_urls):
        self.threat_urls = threat_urls
        # Initialize ChromaDB collection
        self.collection = client.get_or_create_collection("threat_intel")
        self.model = model

    def add_document(self, text, doc_id=None):
        # Embed document and add it to ChromaDB
        embedding = self.model.encode(text, show_progress_bar=False).tolist()
        self.collection.add(
            documents=[text],
            metadatas=[{'source': 'manual'}],
            ids=[doc_id or str(datetime.now())],
            embeddings=[embedding]
        )

    def _search_chroma_db(self, query):
        # Embed query and search in ChromaDB
        query_embedding = self.model.encode(query, show_progress_bar=False).tolist()
        results = self.collection.query(query_embeddings=[query_embedding], n_results=3)
        
        if results['documents']:
            return results['documents'][0]  # Return first list of documents
        return ["No relevant information found in ChromaDB collection."]

    def _clean_text(self, text):
        # Remove HTML entities and unnecessary whitespace
        text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces/newlines with a single space
        text = re.sub(r'&[a-zA-Z0-9#]+;', '', text)  # Remove HTML entities
        return text.strip()

    def _search_urls(self, query):
        # Search through URLs for relevant information
        results = []
        query_embedding = self.model.encode(query, show_progress_bar=False)

        for url in self.threat_urls:
            try:
                response = requests.get(url, timeout=10)
                if response.status_code == 200:
                    soup = BeautifulSoup(response.text, 'html.parser')
                    text = soup.get_text()
                    sentences = text.split('.')
                    embeddings = [self.model.encode(sentence, show_progress_bar=False) for sentence in sentences]
                    
                    # Calculate cosine similarity for each sentence
                    similarities = [
                        torch.nn.functional.cosine_similarity(
                            torch.tensor(query_embedding), torch.tensor(emb), dim=0
                        ).item() for emb in embeddings
                    ]
                    
                    # Get the top 3 most relevant sentences
                    top_indices = sorted(range(len(similarities)), key=lambda i: similarities[i], reverse=True)[:3]
                    relevant_info = [self._clean_text(sentences[i]) for i in top_indices]  # Clean sentences
                    
                    results.append(f"Information from {url}:\n{' '.join(relevant_info)}")
                else:
                    results.append(f"Failed to retrieve content from {url}. Status code: {response.status_code}")
            except Exception as e:
                results.append(f"Error accessing {url}: {str(e)}")
        
        return results or ["No information found from external sources."]

    def search_threat_intel(self, query):
        chroma_results = self._search_chroma_db(query)
        url_results = self._search_urls(query)
        
        return {
            "query": query,
            "timestamp": datetime.now().isoformat(),
            "chroma_db_results": chroma_results,
            "url_results": url_results
        }


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
# Initialize URLs and ChromaDB
threat_urls = [
    "https://fr.wikipedia.org/wiki/Attaque_par_d%C3%A9ni_de_service",   # Example threat intelligence URLs
    "https://www.fortinet.com/resources/cyberglossary/ddos-attack"
]
rag_system = ThreatIntelRAG(threat_urls)

# Add a sample document with relevant information about DDoS attacks
ddos_text = "."
rag_system.add_document(ddos_text, doc_id="ddos_doc_1")

# Query the RAG system with "What is DDoS?"
query = "What is DDoS?"
result = rag_system.search_threat_intel(query)
print(result)


{'query': 'What is DDoS?', 'timestamp': '2024-11-08T18:10:45.940691', 'chroma_db_results': ['.'], 'url_results': ["Information from https://fr.wikipedia.org/wiki/Attaque_par_d%C3%A9ni_de_service:\n↑ (en) Kaspersky Lab, « Denial Of Service: How Businesses Evaluate The Threat Of Ddos Attacks », It Security Risks Special Report Series,\u200e 2015, p On parle alors de DDoS (distributed denial of service attack) Les DDoS (ou attaques DoS Distribuées) seraient plus récentes : la première attaque DDoS connue a eu lieu en août 1999[1] : un outil appelé « Trinoo DDO » (décrit ci-dessous) a été déployé dans au moins 227 systèmes, dont 114 étaient sur Internet, pour inonder les serveurs de l'université du Minnesota", 'Information from https://www.fortinet.com/resources/cyberglossary/ddos-attack:\nDDoS Attacks FAQs What is a DDoS attack? DDoS attack means "Distributed Denial-of-Service (DDoS) attack" and it is a cybercrime in which the attacker floods a server with internet traffic to prevent user